In [ ]:
from pathlib import Path

import pandas as pd
from chess.engine import SimpleEngine

from metrics.metrics import legal, unique, counter_intuitive

In [3]:
base_path = Path(".")
dataset_path = base_path / "dataset" / "dataset.csv"
generation_path = base_path / "generated_fens.txt"
stockfish_path = base_path / ".." / "Stockfish" / "src" / "stockfish"

In [14]:
for chunk in pd.read_csv(dataset_path, chunksize=100_000):
    filtered_df = chunk[chunk["Themes"].str.contains(r'\b{}\b'.format("zugzwang"), regex=True)]
    break
filtered_df.head()

,Unnamed: 0,FEN,Moves,Rating,Themes,Puzzle_FEN
15,15,8/2p1k3/6p1/1p1P1p2/1P3P2/3K2Pp/7P/8 b - - 1 43,e7d6 d3d4 g6g5 f4g5,944,crushing endgame pawnEndgame short zugzwang,8/2p5/3k2p1/1p1P1p2/1P3P2/3K2Pp/7P/8 w - - 2 44
532,532,8/8/5pp1/5k1p/5P1P/4R1P1/2r5/5K2 w - - 0 43,e3e2 c2e2 f1e2 f5g4 e2f2 g4h3,2231,crushing endgame long rookEndgame zugzwang,8/8/5pp1/5k1p/5P1P/6P1/2r1R3/5K2 b - - 1 43
598,598,8/1p6/7p/3k1pp1/1P5P/3K1P2/6P1/8 w - - 0 39,h4g5 h6g5 d3c3 d5e5 c3c4 e5f4 c4c5 f4g3,2709,crushing endgame pawnEndgame veryLong zugzwang,8/1p6/7p/3k1pP1/1P6/3K1P2/6P1/8 b - - 0 39
647,647,8/3k4/1pp3p1/4Pp1p/2PK1P1P/pP6/P7/8 b - - 1 35,d7e6 b3b4 e6e7 b4b5 c6b5 c4b5,2234,crushing endgame long pawnEndgame quietMove zu...,8/8/1pp1k1p1/4Pp1p/2PK1P1P/pP6/P7/8 w - - 2 36
688,688,8/8/3p2N1/3Pp1n1/4Pp1p/3k1P1P/6K1/8 w - - 8 57,g6h4 d3e2 h4f5 g5f3,2109,crushing endgame knightEndgame master short zu...,8/8/3p4/3Pp1n1/4Pp1N/3k1P1P/6K1/8 b - - 0 57


In [ ]:
generations = pd.read_csv(generation_path, header=None, names=["index", "FEN", "rating", "theme", "legal"])
generated_fens = set(generations["FEN"])

overlap = 0
for chunk in pd.read_csv(dataset_path, chunksize=100_000):
    overlap += chunk["Puzzle_FEN"].isin(generated_fens).sum()

print(overlap)

0


In [11]:
n = 100

In [ ]:
n_unique_lichess = 0
with SimpleEngine.popen_uci(stockfish_path) as engine:
    for chunk in pd.read_csv(dataset_path, chunksize=n):
        for i, fen in enumerate(chunk["Puzzle_FEN"]):
            n_unique_lichess += unique(fen, engine)
            if i % 10 == 0: print(i, fen)
        break


0 r6k/pp2r2p/4Rp1Q/3p4/8/1N1P2b1/PqP3PP/7K w - - 0 25
10 5rk1/p5p1/3bRr1p/1Pp4q/3p4/1P1Q1N2/P4PPP/4R1K1 b - - 0 22
20 r2qk2r/pp2ppbp/1n1p2p1/3P4/2n5/2NBBP1P/PP3P2/R2QK2R w KQkq - 0 13
30 8/3B2pp/p5k1/6P1/1ppp1K2/8/1P6/8 w - - 0 39
40 6nr/p4p1p/k1p5/1p6/1QN5/2P1P3/4KPqP/8 w - - 0 27
50 r3brk1/5pp1/p2qpn1p/P2pn3/2pP4/2P1PN2/5PPP/RB1QK2R w KQ - 0 17
60 8/6p1/2B2n2/3b2k1/3B4/6K1/4P3/8 w - - 5 45
70 2nk4/8/2PBp3/1pK1P1p1/1P4Pn/8/8/8 w - - 3 43
80 r4rk1/1pp2ppp/p2p4/2bPp3/2P1PB1q/P1N2B2/1P3P2/R2QK1R1 b Q - 0 15
90 6rk/pp6/2n5/3ppn1p/3p4/2P2P1q/PP3QNB/R5RK b - - 3 29


In [ ]:
n_counter_intuitive_lichess = 0
with SimpleEngine.popen_uci(stockfish_path) as engine:
    for chunk in pd.read_csv(dataset_path, chunksize=n):
        for i, fen in enumerate(chunk["Puzzle_FEN"]):
            n_counter_intuitive_lichess += counter_intuitive(fen, engine)
            if i % 10 == 0: print(i, fen)
        break


0 r6k/pp2r2p/4Rp1Q/3p4/8/1N1P2b1/PqP3PP/7K w - - 0 25
10 5rk1/p5p1/3bRr1p/1Pp4q/3p4/1P1Q1N2/P4PPP/4R1K1 b - - 0 22
20 r2qk2r/pp2ppbp/1n1p2p1/3P4/2n5/2NBBP1P/PP3P2/R2QK2R w KQkq - 0 13
30 8/3B2pp/p5k1/6P1/1ppp1K2/8/1P6/8 w - - 0 39
40 6nr/p4p1p/k1p5/1p6/1QN5/2P1P3/4KPqP/8 w - - 0 27
50 r3brk1/5pp1/p2qpn1p/P2pn3/2pP4/2P1PN2/5PPP/RB1QK2R w KQ - 0 17
60 8/6p1/2B2n2/3b2k1/3B4/6K1/4P3/8 w - - 5 45
70 2nk4/8/2PBp3/1pK1P1p1/1P4Pn/8/8/8 w - - 3 43
80 r4rk1/1pp2ppp/p2p4/2bPp3/2P1PB1q/P1N2B2/1P3P2/R2QK1R1 b Q - 0 15
90 6rk/pp6/2n5/3ppn1p/3p4/2P2P1q/PP3QNB/R5RK b - - 3 29


In [7]:
n_unique_lichess, n_counter_intuitive_lichess

(92, 16)

In [ ]:
generations = pd.read_csv(generation_path, header=None, names=["index", "FEN", "rating", "theme", "legal"], nrows=n)

n_unique_generations = 0
with SimpleEngine.popen_uci(stockfish_path) as engine:
    for i, fen in enumerate(generations["FEN"]):
        if legal(fen): n_unique_generations += unique(fen, engine)
        if i % 10 == 0: print(i, fen)


0 8/p7/1p5p/6p1/1P3kP1/P6P/2N3K1/2b5 w - - 3 45
10 8/6kP/8/1b1p2P1/p4K2/P3PP2/8/8 w - - 1 47
20 r4nk1/p6p/Q1p4r/4P1p1/7q/2P1p1N1/P1P3PP/R4RK1 b - - 4 20
30 1r5k/6pp/p3pr2/q2n4/2pP1P2/B3P3/1R3PQP/2R3K1 b - - 7 26
40 r1bq4/ppppk3/4p2r/1PP3Nn/P2P4/2N5/5PPK/R2Q1R2 b - - 0 17
50 8/1P6/2n1K3/8/1kB5/p6p/7P/8 b - - 4 49
60 8/2k2p2/1R6/2p2r1p/P6P/5nPK/8/8 w - - 1 43
70 2r3k1/p4ppp/8/3R2Q1/1P3P2/7P/r2RKP2/1q6 w - - 2 32
80 5rk1/1Q3p1p/1b2bp2/qB6/P7/6P1/1P3P1P/R3R1K1 b - - 0 24
90 4q3/7k/5QP1/4rPK1/8/8/8/8 w - - 1 70


In [ ]:
generations = pd.read_csv(generation_path, header=None, names=["index", "FEN", "rating", "theme", "legal"], nrows=n)

n_counter_intuitive_generations = 0
with SimpleEngine.popen_uci(stockfish_path) as engine:
    for i, fen in enumerate(generations["FEN"]):
        if legal(fen): n_counter_intuitive_generations += counter_intuitive(fen, engine)
        if i % 10 == 0: print(i, fen)


0 8/p7/1p5p/6p1/1P3kP1/P6P/2N3K1/2b5 w - - 3 45
10 8/6kP/8/1b1p2P1/p4K2/P3PP2/8/8 w - - 1 47
20 r4nk1/p6p/Q1p4r/4P1p1/7q/2P1p1N1/P1P3PP/R4RK1 b - - 4 20
30 1r5k/6pp/p3pr2/q2n4/2pP1P2/B3P3/1R3PQP/2R3K1 b - - 7 26
40 r1bq4/ppppk3/4p2r/1PP3Nn/P2P4/2N5/5PPK/R2Q1R2 b - - 0 17
50 8/1P6/2n1K3/8/1kB5/p6p/7P/8 b - - 4 49
60 8/2k2p2/1R6/2p2r1p/P6P/5nPK/8/8 w - - 1 43
70 2r3k1/p4ppp/8/3R2Q1/1P3P2/7P/r2RKP2/1q6 w - - 2 32
80 5rk1/1Q3p1p/1b2bp2/qB6/P7/6P1/1P3P1P/R3R1K1 b - - 0 24
90 4q3/7k/5QP1/4rPK1/8/8/8/8 w - - 1 70


In [13]:
n_unique_generations, n_counter_intuitive_generations

(22, 45)